# Credit risk for German banks

#### EL = PD * LGD * ED
- EL: Expected Loss
- PD: Probability of Default
- LGD: Loss given default
- ED: Exposure at Default

### Exploratory Data Analysis

In [2]:
import pandas as pd

In [24]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [4]:
df = pd.read_csv('credit.csv')
print(df.shape)

(1000, 21)


In [5]:
df.columns

Index(['Creditability', 'Account Balance', 'Duration of Credit (month)',
       'Payment Status of Previous Credit', 'Purpose', 'Credit Amount',
       'Value Savings/Stocks', 'Length of current employment',
       'Instalment per cent', 'Sex & Marital Status', 'Guarantors',
       'Duration in Current address', 'Most valuable available asset',
       'Age (years)', 'Concurrent Credits', 'Type of apartment',
       'No of Credits at this Bank', 'Occupation', 'No of dependents',
       'Telephone', 'Foreign Worker'],
      dtype='object')

In [6]:
num_records = len(df)
num_records

1000

Check the data types of columns

In [7]:
data_types = df.dtypes
data_types

Creditability                        int64
Account Balance                      int64
Duration of Credit (month)           int64
Payment Status of Previous Credit    int64
Purpose                              int64
Credit Amount                        int64
Value Savings/Stocks                 int64
Length of current employment         int64
Instalment per cent                  int64
Sex & Marital Status                 int64
Guarantors                           int64
Duration in Current address          int64
Most valuable available asset        int64
Age (years)                          int64
Concurrent Credits                   int64
Type of apartment                    int64
No of Credits at this Bank           int64
Occupation                           int64
No of dependents                     int64
Telephone                            int64
Foreign Worker                       int64
dtype: object

In [8]:
def missing_ratio(data):
    return data.isna().mean() * 100

def num_diff_vals(data):
    return data.nunique()

def diff_val(data):
    return data.dropna().unique()

In [9]:
cols_info = df.agg([missing_ratio, pd.Series.max, pd.Series.min])
cols_info

,Creditability,Account Balance,Duration of Credit (month),Payment Status of Previous Credit,Purpose,Credit Amount,Value Savings/Stocks,Length of current employment,Instalment per cent,Sex & Marital Status,Guarantors,Duration in Current address,Most valuable available asset,Age (years),Concurrent Credits,Type of apartment,No of Credits at this Bank,Occupation,No of dependents,Telephone,Foreign Worker
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,1.0,4.0,72.0,4.0,10.0,18424.0,5.0,5.0,4.0,4.0,3.0,4.0,4.0,75.0,3.0,3.0,4.0,4.0,2.0,2.0,2.0
min,0.0,1.0,4.0,0.0,0.0,250.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,19.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


Check for data class imbalance

In [10]:
df['Creditability'].value_counts(normalize=True)

1    0.7
0    0.3
Name: Creditability, dtype: float64

For class data imbalance, I will use two different approach:
- Upsampling for minor class + Logistics regression
- Tree-based statiscal learning algorithm

Check feature values imbalance in columns with categorical values.

In [13]:
numerical_cols = ['Duration of Credit (month)', 'Credit Amount', 'Age (years)']

In [43]:
(pd.DataFrame(
    df.loc[:, ~df.columns.isin(numerical_cols)]
    .melt(var_name='column', value_name='value')
    .groupby(by=['column'])['value'].apply(pd.Series.value_counts, normalize=True))
.sort_values(by=['column', 'value']))

value
column                                     
Account Balance                   3   0.063
                                  2   0.269
                                  1   0.274
                                  4   0.394
Concurrent Credits                2   0.047
                                  1   0.139
                                  3   0.814
Creditability                     0   0.300
                                  1   0.700
Duration in Current address       1   0.130
                                  3   0.149
                                  2   0.308
                                  4   0.413
Foreign Worker                    2   0.037
                                  1   0.963
Guarantors                        2   0.041
                                  3   0.052
                                  1   0.907
Instalment per cent               1   0.136
                                  3   0.157
                                  2   0.231
                                  4   0.476
Length of current employment      1   0.062
                                  2   0.172
                                  4   0.174
                                  5   0.253
                                  3   0.339
Most valuable available asset     4   0.154
                                  2   0.232
                                  1   0.282
                                  3   0.332
No of Credits at this Bank        4   0.006
                                  3   0.028
                                  2   0.333
                                  1   0.633
No of dependents                  2   0.155
                                  1   0.845
Occupation                        1   0.022
                                  4   0.148
                                  2   0.200
                                  3   0.630
Payment Status of Previous Credit 0   0.040
                                  1   0.049
                                  3   0.088
                                  4   0.293
                                  2   0.530
Purpose                           8   0.009
                                  10  0.012
                                  4   0.012
                                  5   0.022
                                  6   0.050
                                  9   0.097
                                  1   0.103
                                  2   0.181
                                  0   0.234
                                  3   0.280
Sex & Marital Status              1   0.050
                                  4   0.092
                                  2   0.310
                                  3   0.548
Telephone                         2   0.404
                                  1   0.596
Type of apartment                 3   0.107
                                  1   0.179
                                  2   0.714
Value Savings/Stocks              4   0.048
                                  3   0.063
                                  2   0.103
                                  5   0.183
                                  1   0.603

There are a huge data imbalance in the "Guarantors" and "Foreign Worker" features => so I will drop these features from the data.

### Data preprocess

Omit features that aren't related to creditability intuitively

In [ ]:
omitted_features = ['Telephone', 'Guarantors', 'Foreign Worker']

## Probability of Default

In [ ]:
def pd(data):
    p